In [1]:
import requests

class ChainbaseAPI:
    def __init__(self, api_key):
        self.base_url = 'https://api.chainbase.online/v1/token/price/history'
        self.api_key = api_key

    def get_price_history(self, chain_id, contract_address, from_timestamp, end_timestamp):
        url = f'{self.base_url}?chain_id={chain_id}&contract_address={contract_address}&from_timestamp={from_timestamp}&end_timestamp={end_timestamp}'
        headers = {'x-api-key': self.api_key, 'accept': 'application/json'}

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()

# Example Usage:
api_key = '2XTnp2w7BC1wSmrf4XHWOwico1o'
chainbase = ChainbaseAPI(api_key)
data = chainbase.get_price_history(1, '0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce', 1672502400, 1677524800)
print(data)


{'code': 0, 'message': 'ok', 'data': [{'price': 1.245e-05, 'symbol': 'shib', 'decimals': 18, 'updated_at': '2023-02-27T19:00:00Z'}, {'price': 1.247e-05, 'symbol': 'shib', 'decimals': 18, 'updated_at': '2023-02-27T18:00:00Z'}, {'price': 1.257e-05, 'symbol': 'shib', 'decimals': 18, 'updated_at': '2023-02-27T17:00:00Z'}, {'price': 1.28e-05, 'symbol': 'shib', 'decimals': 18, 'updated_at': '2023-02-27T16:00:00Z'}, {'price': 1.28e-05, 'symbol': 'shib', 'decimals': 18, 'updated_at': '2023-02-27T15:00:00Z'}, {'price': 1.279e-05, 'symbol': 'shib', 'decimals': 18, 'updated_at': '2023-02-27T14:00:00Z'}, {'price': 1.261e-05, 'symbol': 'shib', 'decimals': 18, 'updated_at': '2023-02-27T13:00:00Z'}, {'price': 1.269e-05, 'symbol': 'shib', 'decimals': 18, 'updated_at': '2023-02-27T12:00:00Z'}, {'price': 1.266e-05, 'symbol': 'shib', 'decimals': 18, 'updated_at': '2023-02-27T11:00:00Z'}, {'price': 1.269e-05, 'symbol': 'shib', 'decimals': 18, 'updated_at': '2023-02-27T10:00:00Z'}, {'price': 1.267e-05, 'sy

In [2]:
import requests
import pandas as pd
import ezkl

class ChainbaseAPI:
    def __init__(self, api_key):
        self.base_url = 'https://api.chainbase.online/v1/token/price/history'
        self.api_key = api_key

    def get_price_history(self, chain_id, contract_address, from_timestamp, end_timestamp):
        url = f'{self.base_url}?chain_id={chain_id}&contract_address={contract_address}&from_timestamp={from_timestamp}&end_timestamp={end_timestamp}'
        headers = {'x-api-key': self.api_key, 'accept': 'application/json'}

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            return response.json()
        else:
            response.raise_for_status()

    def save_to_csv(self, chain_id, contract_address, from_timestamp, end_timestamp, filename):
        data = self.get_price_history(chain_id, contract_address, from_timestamp, end_timestamp)

        if 'data' in data and len(data['data']) > 0:
            data_list = data['data']
            df = pd.DataFrame(data_list)
            df.to_csv(filename, index=False)
            return True
        else:
            return False

# Example Usage:
api_key = '2XTnp2w7BC1wSmrf4XHWOwico1o'
chainbase = ChainbaseAPI(api_key)
success = chainbase.save_to_csv(1, '0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce', 1672502400, 1677524800, 'price_data.csv')

if success:
    print("Data saved to price_data.csv")
else:
    print("No data to save.")


Data saved to price_data.csv


In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('price_data.csv')  # Replace 'your_data.csv' with the actual file path

# Drop any unnecessary columns (if any)
# data = data.drop(columns=['symbol', 'decimals', 'updated_at'])

# Convert the date-time column to datetime format
data['updated_at'] = pd.to_datetime(data['updated_at'])

# Extract features from the date-time column (e.g., hour, minute, etc.)
data['hour'] = data['updated_at'].dt.hour
data['minute'] = data['updated_at'].dt.minute
data['dayofweek'] = data['updated_at'].dt.dayofweek

# Drop the original date-time column
data = data.drop(columns=['updated_at'])

In [13]:
data=pd.read_csv('/content/price_data.csv')

In [17]:
data['price'].head(20)

0     0.000012
1     0.000012
2     0.000013
3     0.000013
4     0.000013
5     0.000013
6     0.000013
7     0.000013
8     0.000013
9     0.000013
10    0.000013
11    0.000013
12    0.000013
13    0.000013
14    0.000013
15    0.000013
16    0.000013
17    0.000013
18    0.000013
19    0.000013
Name: price, dtype: float64

In [19]:
data=data['price']

In [29]:
import pandas as pd
import numpy as np

sequence_length = 3
X = []
y = []

for i in range(len(data) - sequence_length):
    X.append(data[i:i+sequence_length])
    y.append(data[i+sequence_length])

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

X = np.reshape(X, (X.shape[0], sequence_length, 1))


train_ratio = 0.8
train_size = int(len(X) * train_ratio)

X_train = X[:train_size]
y_train = y[:train_size]

X_test = X[train_size:]
y_test = y[train_size:]


In [30]:
import torch
import torch.nn as nn
import numpy as np

# Define the RNN model
class PriceForecastingModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PriceForecastingModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

# Set hyperparameters
input_size = 1  # Number of features (in this case, it's just the price)
hidden_size = 64  # Number of hidden units in the RNN
num_layers = 2  # Number of RNN layers
output_size = 1  # Number of output units (in this case, it's just predicting the next price)

# Create an instance of the model
model = PriceForecastingModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Convert data to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

# Training loop
num_epochs = 1000

for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train)
    optimizer.zero_grad()
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on test data
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

# Predict future prices (you can replace this with your own data)
future_prices = []

# Assuming 'future_data' is the input data for prediction
# Convert 'future_data' to the same format as X (sequences of length 3)

Epoch [100/1000], Loss: 0.0000
Epoch [200/1000], Loss: 0.0000
Epoch [300/1000], Loss: 0.0000
Epoch [400/1000], Loss: 0.0000
Epoch [500/1000], Loss: 0.0000
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000
Test Loss: 0.0000


In [ ]:
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])


except:
    pass

import os
import json
import ezkl

In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.ezkl')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:

x = []


model.eval()

# Export the model
torch.onnx.export(model,
                  x,
                  model_path,
                  export_params=True,
                  opset_version=10,
                  do_constant_folding=True,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes={'input' : {0 : 'batch_size'},
                                'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump(data, open(data_path, 'w'))

In [ ]:
!RUST_LOG=trace

res = ezkl.gen_settings(model_path, settings_path)
assert res == True

res = ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")  # Optimize for resources

In [ ]:
res = ezkl.compile_model(model_path, compiled_model_path, settings_path)
assert res == True

In [ ]:
res = ezkl.get_srs(srs_path, settings_path)

In [ ]:
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        srs_path,
        settings_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [ ]:
# now generate the witness file
witness_path = os.path.join('witness.json')

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path, settings_path = settings_path)
assert os.path.isfile(witness_path)

In [ ]:
# Generate the proof

proof_path = os.path.join('proof.json')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "evm",
        "single",
        settings_path,
    )

print(res)
assert os.path.isfile(proof_path)

In [ ]:
# verify our proof

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")

In [ ]:
try:
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "solc-select"])
    !solc-select install 0.8.20
    !solc-select use 0.8.20
    !solc --version

# rely on local installation if the notebook is not in colab
except:
    pass

In [ ]:
    sol_code_path = os.path.join('Verifier.sol')
    abi_path = os.path.join('Verifier.abi')

    res = ezkl.create_evm_verifier(
        vk_path,
        srs_path,
        settings_path,
        sol_code_path,
        abi_path
    )

    assert res == True
    assert os.path.isfile(sol_code_path)

In [ ]:
# Get the sample inputs from the proof file
with open("proof.json", "r") as f:
    proof = json.load(f)

In [ ]:
onchain_input_array = []

for value in proof["instances"]:
    for field_element in value:
        onchain_input_array.append(ezkl.vecu64_to_int(field_element))
# This will be the values you use onchain
# copy them over to remix and see if they verify
# What happens when you change a value?
print("pubInputs: ", onchain_input_array)
print("proof: ", "0x" + proof["proof"])